In [ ]:
import numpy as np
from flask import Flask, request, jsonify, render_template,send_file, make_response
import pickle
from sklearn.utils import shuffle
import pandas as pd
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import io
import seaborn as sns




def ROI_total(loans):
    weighted_term_og = loans['term_amnt'].sum() / loans['loan_amnt'].sum()
    weighted_interest_og = loans['int_amnt'].sum() / loans['loan_amnt'].sum()
    charge_off_rate_og = loans['loan_class'].value_counts(normalize=True)
    charge_off_amount_og = loans.groupby('loan_class').loan_amnt.sum() #dollar amount of each loan_class
    total_CoR_og = charge_off_amount_og[1] / loans['loan_amnt'].sum() #dollar amount of default loans

    loans_approved_og = loans.loan_amnt.sum() #total dollar amount of loans 
    loans_charged_off_og = charge_off_amount_og[1] #total dollar amount of default loans

    # And therefore, the RoE:
    loans_returned_og = loans_approved_og - loans_charged_off_og #total loan amount paid back
    interest_earned_og = loans_returned_og * (weighted_interest_og) / 100 * (weighted_term_og / 12) 
    interest_paid_og = loans_approved_og * CoF / 100 * (weighted_term_og / 12) #CoF (cost of funds)
    profit_og = interest_earned_og - interest_paid_og - loans_charged_off_og
    ROI2 = profit_og / loans_approved_og
    return ROI2

def ROI_model(n):
    COF = 3
    a, b, c, d, e, f, g = n[0],n[1],n[2],n[3],n[4],n[5],n[6]
    loans_test['column'] = y_pred
    y_test_paid = loans_test.loc[loans_test['column'].isin([0])]
    a_loans = shuffle(y_test_paid.loc[(y_test_paid['sub_grade'] >= 1) & (y_test_paid['sub_grade'] <= 5)])[0:a]
    b_loans = shuffle(y_test_paid.loc[(y_test_paid['sub_grade'] >= 6) & (y_test_paid['sub_grade'] <= 10)])[0:b]
    c_loans = shuffle(y_test_paid.loc[(y_test_paid['sub_grade'] >= 11) & (y_test_paid['sub_grade'] <= 15)])[0:c]
    d_loans = shuffle(y_test_paid.loc[(y_test_paid['sub_grade'] >= 16) & (y_test_paid['sub_grade'] <= 20)])[0:d]
    e_loans = shuffle(y_test_paid.loc[(y_test_paid['sub_grade'] >= 21) & (y_test_paid['sub_grade'] <= 25)])[0:e]
    f_loans = shuffle(y_test_paid.loc[(y_test_paid['sub_grade'] >= 26) & (y_test_paid['sub_grade'] <= 30)])[0:f]
    g_loans = shuffle(y_test_paid.loc[(y_test_paid['sub_grade'] >= 31) & (y_test_paid['sub_grade'] <= 35)])[0:g]
    
    y_test_paid = pd.concat([a_loans, b_loans, c_loans, d_loans, e_loans, f_loans, g_loans])   
    performance = y_test_paid.groupby('loan_class').loan_amnt.sum()
    
    # CoR is the proportion that was charged off:
    loans_approved = performance.sum()
    loans_charged_off = performance[1]
    CoR = loans_charged_off / loans_approved

    saving = loans_test['loan_amnt'].sum() * (total_CoR_test - CoR)

    weighted_interest = y_test_paid['int_amnt'].sum() / y_test_paid['loan_amnt'].sum()
    weighted_term = y_test_paid['term_amnt'].sum() / y_test_paid['loan_amnt'].sum()

    # ROI
    loans_returned = loans_approved - loans_charged_off
    interest_earned = loans_returned * weighted_interest / 100 * (weighted_term / 12)
    interest_paid = loans_approved * CoF / 100 * (weighted_term / 12)
    profit = interest_earned - interest_paid - loans_charged_off
    ROI = (profit / loans_approved)
    
    output = round(ROI, 2)
    return output

def simulation(n):
    model_ROI = []
    lc_ROI = []

    for i in range(1000):
        loan_shuffled = shuffle(loans)
        l_ROI = ROI_total(loan_shuffled[0:1000])
        lc_ROI.append(l_ROI)
    
        m_ROI = ROI_model(n)
        model_ROI.append(m_ROI)
    return model_ROI, lc_ROI

app = Flask(__name__)
loans = pickle.load(open('cleaned_data.pkl', 'rb'))
loans_test = pickle.load(open('loans_test.pkl', 'rb'))
total_CoR_test = 0.1803
CoF = 3
y_pred = pickle.load(open('y_pred_xg.pkl', 'rb'))
loans = loans

@app.route('/')
def home():
    return render_template('index.html')


@app.route('/predict',methods=['POST'])
def predict():

    n = [int(x) for x in request.form.values()]

    output = ROI_model(n)
    model_ROI, lc_ROI = simulation(n)
    
    ROI_df = pd.DataFrame()
    ROI_df['model_ROI'] = model_ROI
    ROI_df['lc_ROI'] = lc_ROI
    
    avg_modelROI = round(np.mean(model_ROI),2)
    avg_lcROI = round(np.mean(lc_ROI),2)
    
    fig, ax = plt.subplots(figsize=(14,7))
    ax=sns.set(style="darkgrid")
    ax = sns.distplot(tuple(ROI_df['model_ROI']), bins = 10)
    ax = sns.distplot(tuple(ROI_df['lc_ROI']), bins = 10)
    ax.set_xlabel("Rate of Return",fontsize=16)
    ax.set_ylabel("Frequency",fontsize=16)
    plt.legend(labels=["Model ROI: {}".format(avg_modelROI), 'Benchmark (Lending Club) ROI: {}'.format(avg_lcROI)], fontsize=16)
    plt.savefig(fname='ROI_Hist', dpi=150)
    
    canvas=FigureCanvas(fig)
    img = io.BytesIO()
    fig.savefig(img)
    img.seek(0)
    print (np.mean(model_ROI))
    return send_file(img, mimetype='image/png')
    
#     return np.mean(model_ROI)
#     return np.mean(lc_ROI)

if __name__ == "__main__":
    app.run(debug=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Apr/2021 11:46:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2021 11:46:33] "GET /predict HTTP/1.1" 405 -
/Users/thomaskim/opt/anaconda3/lib/python3.8/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/Users/thomaskim/opt/anaconda3/lib/python3.8/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
127.0.0.1 - - [16/Apr/2021 11:52:20] "POST /predict HTTP/1.1" 200 -


0.1444


127.0.0.1 - - [16/Apr/2021 12:18:56] "GET /predict HTTP/1.1" 405 -
/Users/thomaskim/opt/anaconda3/lib/python3.8/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/Users/thomaskim/opt/anaconda3/lib/python3.8/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
127.0.0.1 - - [16/Apr/2021 12:24:42] "POST /predict HTTP/1.1" 200 -


0.14532


127.0.0.1 - - [16/Apr/2021 15:10:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2021 15:10:51] "GET /predict HTTP/1.1" 405 -


In [12]:
%tb

SystemExit: 1

In [51]:
#     fig, ax = plt.subplots(figsize=(14,7))
#     ax=sns.set(style="darkgrid")
#     ax = sns.distplot(tuple(ROI_df['model_ROI']))
#     ax = sns.distplot(tuple(ROI_df['lc_ROI']))
#     ax.set_xlabel("Rate of Return",fontsize=16)
#     ax.set_ylabel("Frequency",fontsize=16)
#     plt.legend(labels=["Model ROI", 'Benchmark (Lending Club) ROI'], fontsize=16)
#     plt.savefig(fname='ROI_Hist', dpi=150)
#     canvas=FigureCanvas(fig)
#     img = io.BytesIO()
#     fig.savefig(img)
#     img.seek(0)
#     return img

#     return render_template('index.html', prediction_text='ROI Prediction is {}'.format(output))


127.0.0.1 - - [08/Apr/2021 01:55:17] "GET / HTTP/1.1" 200 -


In [ ]:
# def mpl():
#     return render_template('matplot.html',
#                           PageTitle = 'Matplotlib')
# @app.route('/plot.png')
# def plot_png():
#     fig = create_figure()
#     output = io.BytesIO()
#     FigureCanvas(fig).print_png(output)
#     return Response(output.getvalue(), mimetype='image/png')

# def create_figure():
#     fig, ax = plt.subplots(figsize=(14,7))
#     ax=sns.set(style="darkgrid")
#     ax = sns.distplot(tuple(ROI_df['model_ROI']))
#     ax = sns.distplot(tuple(ROI_df['lc_ROI']))
#     ax.set_xlabel("Rate of Return",fontsize=16)
#     ax.set_ylabel("Frequency",fontsize=16)
#     plt.legend(labels=["Model ROI", 'Benchmark (Lending Club) ROI'], fontsize=16)
#     plt.savefig(fname='ROI_Hist', dpi=150)
    
#     return fig